In [ ]:
import json

import polars as pl

from src.lib import version_extractor, name_extractor

In [ ]:
df = pl.read_json('../../data/big_query/bq_results.json', json_lines=True)

In [ ]:
df_sorted = df.sort(['name', 'version', 'upload_time'], reverse=[False, True, False])
print(df_sorted)

In [ ]:
# Eliminate duplicates
df_unique = df_sorted.unique(subset=['name', 'version'])
print(df_unique)

In [ ]:
# Used to find the true number of packages
print(len(df_unique.groupby('name').groups()))

In [ ]:
data = json.loads(df_unique.to_pandas().to_json(orient='records'))

In [ ]:
def extract_name_and_version(dependency_version_string: str) -> (str, str):
    dep_name = name_extractor(dependency_version_string)
    dep_version = version_extractor(dependency_version_string)
    if dep_name and version:
        return dep_name, dep_version

results: dict[str, dict[str, dict]] = {}

for index, dictionary in enumerate(data):
    name = dictionary['name']
    version = dictionary['version']
    upload_time = dictionary['upload_time']

    if name not in results:
        normalized_form = {
            'name': name,
            'versions': {}
        }
    else:
        normalized_form = results.get(name)

    normalized_form['versions'][version] = {
        'timestamp': upload_time,
        'dependencies': {}
    }

    for dep in dictionary['requires_dist']:

        dependency_name, dependency_version = extract_name_and_version(dep)
        normalized_form['versions'][version]['dependencies'][dependency_name] = dependency_version
    results[name] = normalized_form

del data
results

In [ ]:
final_result = {'packages': list(results.values())}
# Uncomment next line to use the old format for the JSON file
# final_result = list(results.values())

with open('../../data/output/pypi-bq-dependencies.json', 'w') as file:
    json.dump(final_result, file)

In [ ]:
# Used to find the true number of packages
print(len(df_unique.groupby('name').groups()))

In [5]:
data = json.loads(df_unique.to_pandas().to_json(orient='records'))

In [6]:
def extract_name_and_version(dependency_version_string: str) -> (str, str):
    dep_name = name_extractor(dependency_version_string)
    dep_version = version_extractor(dependency_version_string)
    if dep_name and version:
        return dep_name, dep_version

results: dict[str, dict[str, dict]] = {}

for index, dictionary in enumerate(data):
    name = dictionary['name']
    version = dictionary['version']
    upload_time = dictionary['upload_time']

    if name not in results:
        normalized_form = {
            'name': name,
            'versions': {}
        }
    else:
        normalized_form = results.get(name)

    normalized_form['versions'][version] = {
        'timestamp': upload_time,
        'dependencies': {}
    }

    for dep in dictionary['requires_dist']:

        dependency_name, dependency_version = extract_name_and_version(dep)
        normalized_form['versions'][version]['dependencies'][dependency_name] = dependency_version
    results[name] = normalized_form

del data
results

{'0': {'name': '0',
  'versions': {'0.0.0': {'timestamp': '2017-08-06 03:43:27.195869 UTC',
    'dependencies': {}}}},
 '0-core-client': {'name': '0-core-client',
  'versions': {'1.1.0a8': {'timestamp': '2017-10-11 13:19:49.81603 UTC',
    'dependencies': {'redis': '>=2.10.5'}},
   '1.1.0a7': {'timestamp': '2017-08-08 18:10:31.426796 UTC',
    'dependencies': {'redis': '>=2.10.5'}},
   '1.1.0a5': {'timestamp': '2017-07-17 06:41:55.349782 UTC',
    'dependencies': {'redis': '>=2.10.5'}},
   '1.1.0a4': {'timestamp': '2017-06-21 15:00:03.092239 UTC',
    'dependencies': {}},
   '1.1.0a3': {'timestamp': '2017-06-09 08:34:43.85207 UTC',
    'dependencies': {'redis': '>=2.10.5'}}}},
 '0-orchestrator': {'name': '0-orchestrator',
  'versions': {'1.1.0a8': {'timestamp': '2017-10-11 13:21:50.027615 UTC',
    'dependencies': {'Jinja2': '>=0.0.0',
     'aioredis': '>=0.0.0',
     'etcd3': '>=0.0.0',
     'netaddr': '>=0.0.0',
     'python-dateutil': '>=0.0.0'}},
   '1.1.0a7': {'timestamp': '2017-0

In [10]:
with open('../../data/output/pypi-bq-dependencies.json', 'w') as file:
    json.dump(list(results.values()), file)